In [3]:
import requests

In [35]:
API_KEY = '64e1315db7fb4cdcba99f9141aacc7c2'
AUDIO_FILE = 'SDS_test_podcast_7min.mp3'

In [36]:
import sys
# from configure import auth_key
import requests
import pprint
from time import sleep
 
# store global constants
headers = {
   "authorization": API_KEY,
   "content-type": "application/json"
}
transcript_endpoint = "https://api.assemblyai.com/v2/transcript"
upload_endpoint = 'https://api.assemblyai.com/v2/upload'
 
# make a function to pass the mp3 to the upload endpoint
def read_file(filename):
   with open(filename, 'rb') as _file:
       while True:
           data = _file.read(5242880)
           if not data:
               break
           yield data
 
# upload our audio file
upload_response = requests.post(
   upload_endpoint,
   headers=headers, data=read_file(AUDIO_FILE)
)
print('Audio file uploaded')
 
# send a request to transcribe the audio file
transcript_request = {'audio_url': upload_response.json()['upload_url']}
transcript_response = requests.post(transcript_endpoint, json=transcript_request, headers=headers)
print('Transcription Requested')
pprint.pprint(transcript_response.json())
# set up polling
polling_response = requests.get(transcript_endpoint+"/"+transcript_response.json()['id'], headers=headers)
filename = transcript_response.json()['id'] + '.txt'
# if our status isn’t complete, sleep and then poll again
while polling_response.json()['status'] != 'completed':
   sleep(30)
   polling_response = requests.get(transcript_endpoint+"/"+transcript_response.json()['id'], headers=headers)
   print("File is", polling_response.json()['status'])
with open(filename, 'w') as f:
   f.write(polling_response.json()['text'])
print('Transcript saved to', filename)

Audio file uploaded
Transcription Requested
{'acoustic_model': 'assemblyai_default',
 'audio_duration': None,
 'audio_end_at': None,
 'audio_start_from': None,
 'audio_url': 'https://cdn.assemblyai.com/upload/ad711c92-e499-44fc-9aad-30ac567495ea',
 'auto_chapters': False,
 'auto_highlights': False,
 'auto_highlights_result': None,
 'boost_param': None,
 'chapters': None,
 'confidence': None,
 'content_safety': False,
 'content_safety_labels': {},
 'disfluencies': False,
 'dual_channel': None,
 'entities': None,
 'entity_detection': False,
 'filter_profanity': False,
 'format_text': True,
 'iab_categories': False,
 'iab_categories_result': {},
 'id': 'oza0sl28ht-9dcd-4365-984b-c609a53046ec',
 'language_code': 'en_us',
 'language_model': 'assemblyai_default',
 'punctuate': True,
 'redact_pii': False,
 'redact_pii_audio': False,
 'redact_pii_audio_quality': None,
 'redact_pii_policies': None,
 'redact_pii_sub': None,
 'sentiment_analysis': False,
 'sentiment_analysis_results': None,
 'spe

In [8]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest

In [31]:
def summarize(summary_file, per):
    nlp = spacy.load('en_core_web_sm')
    with open (summary_file, "r") as myfile:
        text = myfile.read()
    doc = nlp(text)
    tokens=[token.text for token in doc]
    word_frequencies={}
    for word in doc:
        if word.text.lower() not in list(STOP_WORDS):
            if word.text.lower() not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1
    max_frequency=max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word]=word_frequencies[word]/max_frequency
    sentence_tokens= [sent for sent in doc.sents]
    sentence_scores = {}
    for sent in sentence_tokens:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if sent not in sentence_scores.keys():                            
                    sentence_scores[sent]=word_frequencies[word.text.lower()]
                else:
                    sentence_scores[sent]+=word_frequencies[word.text.lower()]
    select_length=int(len(sentence_tokens)*per)
    summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
    final_summary=[word.text for word in summary]
    summary=''.join(final_summary)
    return text, summary

In [37]:
text, summary = summarize('SDS_test_podcast_7min.txt', per=0.1)

In [4]:
import streamlit as st

uploaded_file = st.file_uploader("Upload your podcast audio file")

In [5]:
st.set_page_config(
    page_title="A/B Testing App", page_icon="📊", initial_sidebar_state="expanded"
)